In [ ]:
import pandas as pd
import numpy as np

In [ ]:
https://www.mdpi.com/1999-4893/17/3/112
Exploratory Data Analysis and Searching Cliques in Graphs
by András Hubai, Sándor Szabó and Bogdán Zaválnij


### Method: Linear Subgroup Detection via Graph-Based Clique Finding

This notebook implements a novel graph-based algorithm to identify **linear subgroups** within a dataset. Unlike traditional clustering methods like K-Means or DBSCAN which find dense, spherical groups, this technique is specifically designed to detect sets of data points that are geometrically aligned (i.e., they lie on a straight line or a flat plane in the feature space).

---

### How It Works ⚙️

The algorithm discovers these structures through a multi-step process:

1.  **Calculate Pairwise Distances:** First, a distance matrix is computed for all pairs of points. It uses **Gower's distance** or other pairwise methods for distance to correctly handle datasets with a mix of numerical and categorical features.

3.  **Identify "Flat" Triangles:** The core of the method is a geometric test. For every possible triplet of points (i, j, k), it checks if the triangle they form is "flat." A triangle is considered flat if its smallest height is less than a tiny threshold, `epsilon`. This is a proxy for determining if three points are nearly co-linear.

4.  **Construct an Auxiliary Graph:** An auxiliary graph `G` is built where:
    * The **nodes** are not the data points themselves, but every unique *pair* of data points (e.g., (i, j)).
    * An **edge** is created between two nodes—say, `(i, j)` and `(j, k)`—if the points `i`, `j`, and `k` form a flat triangle.

5.  **Find the Largest Clique:** The algorithm then finds the largest **clique** in this auxiliary graph. A clique is a subset of nodes where every node is connected to every other node. In this context, the largest clique represents the largest set of point-pairs that all form flat triangles with each other. The original data points involved in this clique form the final linear subgroup.

---

### When to Use This Method ✅

This technique is highly specialized. It is most powerful in scenarios where you have a hypothesis that a subset of your data follows a simple, linear rule.

**Ideal use cases include:**

* **Progression Analysis:** Identifying subjects whose condition (e.g., disease progression) or behavior (e.g., product adoption) changes at a **constant, predictable rate** over time.
* **Mixture/Formulation Data:** Finding the effect of systematically changing ingredient proportions in a mixture, such as in material science or chemical formulation, where results are expected to follow a linear trade-off.
* **Rule Validation & Anomaly Detection:** In operational data (e.g., shipping logistics, finance), you can identify the subgroup that perfectly follows a known pricing or cost model. The data points *not* in this group are interesting anomalies worth investigating.

**Key Consideration:** This method is sensitive to the `epsilon` parameter and is computationally expensive (O(n³)). It is best used for targeted exploration rather than general-purpose clustering.

In [ ]:
def calculate_triangle_area(d_ij, d_jk, d_ki):
    """
    Calculates the area of a triangle using Heron's formula from its side lengths.

    Args:
        d_ij (float): The distance between point i and point j.
        d_jk (float): The distance between point j and point k.
        d_ki (float): The distance between point k and point i.

    Returns:
        float: The area of the triangle.
    """
    # Calculate the semi-perimeter of the triangle
    s = (d_ij + d_jk + d_ki) / 2.0
    
    # Use max(0, ...) to prevent math errors from floating point inaccuracies
    area_squared = s * max(0, s - d_ij) * max(0, s - d_jk) * max(0, s - d_ki)
    
    return np.sqrt(area_squared)

def is_triangle_flat(p1_idx, p2_idx, p3_idx, D, epsilon):
    """
    Checks if a triangle defined by three points is "flat" by checking if its 
    smallest height is less than a threshold (epsilon).

    Args:
        p1_idx (int): The index of the first point.
        p2_idx (int): The index of the second point.
        p3_idx (int): The index of the third point.
        D (np.ndarray): The MxM pairwise distance matrix.
        epsilon (float): The flatness threshold.

    Returns:
        bool: True if the triangle is flat, False otherwise.
    """
    # Get the lengths of the triangle's sides from the distance matrix
    distances = [D[p1_idx, p2_idx], D[p2_idx, p3_idx], D[p3_idx, p1_idx]]
    
    # A triangle with a side of zero length is degenerate and thus perfectly flat
    if any(d == 0 for d in distances):
        return True

    # Find the longest side of the triangle
    delta = max(distances)
    
    # Calculate the area of the triangle
    area = calculate_triangle_area(distances[0], distances[1], distances[2])
    
    # The smallest height of the triangle is calculated as 2 * Area / longest_side
    mu = 2 * area / delta
    
    # The triangle is flat if its smallest height is less than or equal to epsilon
    return mu <= epsilon

def is_quadrangle_flat(p_idx, q_idx, r_idx, s_idx, D, epsilon):
    """
    Checks if a quadrangle is "flat" by verifying that all four of its 
    constituent triangles are also flat.

    Args:
        p_idx (int): The index of the first point.
        q_idx (int): The index of the second point.
        r_idx (int): The index of the third point.
        s_idx (int): The index of the fourth point.
        D (np.ndarray): The MxM pairwise distance matrix.
        epsilon (float): The flatness threshold.

    Returns:
        bool: True if the quadrangle is flat, False otherwise.
    """
    # A quadrangle is flat if and only if all four triangles inside it are flat.
    # Check each of the four possible triangles.
    if not is_triangle_flat(p_idx, q_idx, r_idx, D, epsilon):
        return False
    if not is_triangle_flat(p_idx, q_idx, s_idx, D, epsilon):
        return False
    if not is_triangle_flat(p_idx, r_idx, s_idx, D, epsilon):
        return False
    if not is_triangle_flat(q_idx, r_idx, s_idx, D, epsilon):
        return False
        
    # If all four triangles passed the test, the quadrangle is flat.
    return True

In [ ]:
import itertools
# The flatness threshold. Since Gower's distance is on a [0, 1] scale,
# this value must be very small. This is the main parameter to tune.
epsilon = 0.01
# Set to False to speed up computation significantly
check_quadrangles = False 

# --- Calculate Gower's Distance Matrix ---
# This correctly handles the mixed numerical and categorical data.
print("Calculating Gower's distance matrix...")

m = len(dfe)
# --- Initialize the Auxiliary Graph ---
print("Initializing the auxiliary graph...")
G = nx.Graph()
# The nodes of the graph are all the unique pairs of points from the dataset.
nodes = list(itertools.combinations(range(m), 2))
G.add_nodes_from(nodes)

# --- Build Graph from Triangles ---
print("Building graph based on triangle flatness (this may take a while)...")
for i, j, k in itertools.combinations(range(m), 3):
    if is_triangle_flat(i, j, k, D, epsilon):
        # If the triangle is flat, connect the corresponding nodes in the graph
        G.add_edge((min(i,j), max(i,j)), (min(j,k), max(j,k)))
        G.add_edge((min(j,k), max(j,k)), (min(k,i), max(k,i)))
        G.add_edge((min(k,i), max(k,i)), (min(i,j), max(i,j)))

# --- (Optional) Build Graph from Quadrangles ---
if check_quadrangles:
    print("Building graph based on quadrangle flatness (this is very slow)...")
    for p, q, r, s in itertools.combinations(range(m), 4):
        if is_quadrangle_flat(p, q, r, s, D, epsilon):
            G.add_edge(tuple(sorted((p,q))), tuple(sorted((r,s))))
            G.add_edge(tuple(sorted((p,r))), tuple(sorted((q,s))))
            G.add_edge(tuple(sorted((p,s))), tuple(sorted((q,r))))

# --- Find and Report the Largest Clique ---
print("Finding the largest clique...")
# nx.find_cliques finds all maximal cliques. We then find the largest one.
all_cliques = list(nx.find_cliques(G))

if not all_cliques:
    print("\nNo cliques found. Try increasing the epsilon threshold.")
else:
    largest_clique_nodes = max(all_cliques, key=len)
    
    # Extract the unique point indices from the nodes of the clique
    object_indices = set()
    for node_pair in largest_clique_nodes:
        object_indices.add(node_pair[0])
        object_indices.add(node_pair[1])
    
    print("\n--- RESULTS ---")
    print(f"Identified a linear subgroup composed of {len(object_indices)} points.")
    print("The indices of these points are:")
    print(sorted(list(object_indices)))
    
    # Display the identified subgroup from the original DataFrame
    print("\nThe data points in this subgroup are:")
    print(df.iloc[sorted(list(object_indices))])